In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from math import inf
from optical_flow import *
from tqdm import tqdm
from tracking_utils import *
import matplotlib.animation as animation
import pickle as pkl
from load_utils import *


In [2]:
# Load paths
detection_path = 'retinanet_101_detections_trained.pkl'
data_path = '../../AICity_data/train/S03/c010/'

gt_detect = readDetectionsXML('ai_challenge_s03_c010-full_annotation.xml')
gt_notParked = getNotParkedCars(gt_detect)
# Load detections
with open(detection_path , 'rb') as f:
    all_detections = pkl.load(f)

In [6]:
def update_track2(detections_pd, next_detections_pd, tolerance=0.5, imgs=None):
    detections_pd['updated'] = False
    detections_pd = detections_pd.reset_index(drop=True)
    
    # Loop each new detection
    for index, next_detection in next_detections_pd.iterrows():
        length, _ = detections_pd.shape
        
        # Find overlaps with max IoU and update if found
        IoUlist = []
        for i in range(length):
            IoU = next_detection['detection'].IoU(detections_pd.iloc[i]['detection'])
            IoUlist.append(IoU)
            
        indexMax = IoUlist.index(max(IoUlist))
            
        if max(IoUlist) > tolerance and detections_pd.at[indexMax,'updated'] != True:
            detections_pd.at[indexMax,'detection'] = next_detection['detection']
            detections_pd.at[indexMax,'bbox'] = next_detection['bbox']
            detections_pd.at[indexMax,'size'] = next_detection['size']
            detections_pd.at[indexMax,'line'].append(next_detection['line'][0])
            detections_pd.at[indexMax,'updated'] = True
            next_detections_pd.at[index, 'updated'] = True
    
    #if images is not None, use optical flow approach to improve tracking
    if imgs is not None:
        idxs = detections_pd.index[detections_pd['updated'] == False].tolist()
        for ind in idxs:
            box = detections_pd.at[ind, 'bbox']
            block = imgs[0][int(box[1]):int(box[3]), int(box[0]):int(box[2])]
            #Compute optical flow of the not updated tracking bounding box
            of = compute_block_of(block, box, imgs[1], 'backward', 10)
            #update bounding box
            if of[0] != 0.0 or of[1] != 0.0:
                newBox = [box[0]+of[0], box[1]+of[1], box[2]+of[0], box[3]+of[1]]
                detections_pd.at[ind, 'bbox'] = newBox
                detections_pd.at[ind,'updated'] = True
                detections_pd.at[indexMax,'detection'].updateBBox(newBox)
                detections_pd.at[indexMax,'line'].append(get_box_center(newBox))

    # Drop detections no longer exist
    detections_pd = detections_pd[detections_pd['updated'] == True]
                
    # Start tracking new detections
    counter = 0
    if any(next_detections_pd['updated'] == False):
        new_pd = next_detections_pd[next_detections_pd['updated']==False]
        new_pd = new_pd.reset_index(drop=True)
        
        # Generate new track number      
        for i in range(len(new_pd)):
            while counter in detections_pd['track'].tolist():
                counter = counter + 1
            new_pd.at[i, 'track'] = counter
            counter = counter + 1

        # Add new tracks
        detections_pd = pd.concat([detections_pd, new_pd])
            
    detections_pd = detections_pd.reset_index(drop=True)
    return detections_pd

In [ ]:
display = False
gif = True

if gif:
    fig, ax = plt.subplots()
    plt.axis('off')

ims = []
detection_history = []

# Get the First Frame
detections_pd = get_detection_dataframe(all_detections['0'])

# Pre-process the first frame if needed
detections_pd = remove_overlaps(detections_pd, 0.9)
detection_history.append(detections_pd)

# Load video
vidcap = cv2.VideoCapture(data_path + 'vdo.avi')
if gif:
    vidcap.set(1,50)
    
num_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))

success, past_frame = vidcap.read()

for frame in tqdm(range(1,50)):
    if not success:
        print('Error reading the video!!')
        break

    success, image = vidcap.read()
    
    next_detections_pd = get_detection_dataframe(all_detections[str(frame)])
    next_detections_pd = remove_overlaps(next_detections_pd, 0.9)
    detections_pd = update_track2(detections_pd, next_detections_pd, tolerance=0.5, imgs=[past_frame, image])
    detection_history.append(detections_pd)
    past_frame = image
    for index, row in detections_pd.iterrows():
        image = drawTrackingOnImage(image, row['bbox'], track=row['track'], line=row['line'],colour=row['colour'])

    if gif:
        im = ax.imshow(image, animated=True)
        ims.append([im])

if gif:
    ani = animation.ArtistAnimation(fig, ims, interval=10, blit=True, repeat_delay=10000)
    ani.save('all_detects' + ".gif", writer=animation.PillowWriter(fps=24))
    
# Open a file and use dump()
# with open('tracking_history/tracking_history.pkl', 'wb') as file:
#     # A new file will be created
#     pkl.dump(detection_history, file)

: 